In [1]:
### starting spark session ###
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1638298316469_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 1. Extracting data from HDFS

In [2]:
### Importing required types ###
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType, DecimalType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
### Constructing the schema names ###
fileSchema = StructType([
    StructField('year', IntegerType(), True),
    StructField('month', StringType(), True),
    StructField('day', IntegerType(), True),
    StructField('weekday', StringType(), True),
    StructField('hour', IntegerType(), True),
    StructField('atm_status', StringType(), True),
    StructField('atm_id', StringType(), True),
    StructField('atm_manufacturer', StringType(), True),
    StructField('atm_location', StringType(), True),
    StructField('atm_street_name', StringType(), True),
    StructField('atm_street_number', IntegerType(), True),
    StructField('atm_zipcode', IntegerType(), True),
    StructField('atm_lat', DecimalType(), True),
    StructField('atm_lon', DecimalType(), True),
    StructField('currency', StringType(), True),
    StructField('card_type', StringType(), True),
    StructField('transaction_amount', IntegerType(), True),
    StructField('service', StringType(), True),
    StructField('message_code', StringType(), True),
    StructField('message_text', StringType(), True),
    StructField('weather_lat', DecimalType(), True),
    StructField('weather_lon', DecimalType(), True),
    StructField('weather_city_id', IntegerType(), True),
    StructField('weather_city_name', StringType(), True),
    StructField('temp', DecimalType(), True),
    StructField('pressure', IntegerType(), True),
    StructField('humidity', IntegerType(), True),
    StructField('wind_speed', IntegerType(), True),
    StructField('wind_deg', IntegerType(), True),
    StructField('rain_3h', DecimalType(), True),
    StructField('clouds_all', IntegerType(), True),
    StructField('weather_id', IntegerType(), True),
    StructField('weather_main', StringType(), True),
    StructField('weather_description', StringType(), True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
### Reading data from csv files ###
sourceFile = spark.read.csv("part-m-00000",
                            multiLine=True,
                            escape="\'",
                            schema=fileSchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
### Verifying the count ###
sourceFile.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [6]:
### displaying the colums ###
sourceFile.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'atm_id', 'atm_manufacturer', 'atm_location', 'atm_street_name', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description']

In [7]:
### Creating temp table for using in spark Sql###
sourceFile.createOrReplaceTempView("sourceFile_SQl")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2. Dimension tables creation 

#### DIM_LOCATION ####


In [8]:
### Identifying the distinct location data using spark sql###

Dim_Location_distinct = spark.sql(
    " select distinct atm_location,atm_street_name,atm_street_number,atm_zipcode,atm_lat,atm_lon from sourceFile_SQl"
)

### Creating temp table with spark Sql ###

Dim_Location_distinct.createOrReplaceTempView("Dim_Location_distinct_sql")

### Generating Row_number as location_id using spark SQl ###

Dim_Location = spark.sql(
    " select ROW_NUMBER() OVER (ORDER BY atm_location) as location_id, atm_location,atm_street_name,atm_street_number,atm_zipcode,atm_lat,atm_lon from Dim_Location_distinct_sql"
)

### Creating temp table with spark Sql to use it in differen join ###

Dim_Location.createOrReplaceTempView("Dim_Location_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
### Writing the Location dimension into S3
Dim_Location.write.format('csv').option('header', 'true').save(
    's3://de-assignment-gov/dim_location/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### DIM_ATM ####

In [10]:
### ATM dimension ###
atm_distinct = spark.sql(
    " select distinct atm_id as atm_number, atm_manufacturer,atm_location,atm_street_name,atm_street_number,atm_zipcode,atm_lat,atm_lon from sourceFile_SQl"
)
### Creating temp table with spark Sql ###
atm_distinct.createOrReplaceTempView("atm_distinct_sql")

### Joining with location temp table ###
atm_location = spark.sql(
    "select distinct a.atm_number,a.atm_manufacturer,b.location_id from  atm_distinct_sql a, Dim_Location_sql b where a.atm_location= b.atm_location and a.atm_street_name= b.atm_street_name and a.atm_street_number=b.atm_street_number and a.atm_zipcode =b.atm_zipcode and a.atm_lat=b.atm_lat and a.atm_lon = b.atm_lon"
)

### Creating temp table to generate primary key ###
atm_location.createOrReplaceTempView("atm_location_sql")

### Generating primary key ###
dim_atm = spark.sql(
    " select ROW_NUMBER() OVER (ORDER BY atm_number) as atm_id, atm_number,atm_manufacturer, location_id from atm_location_sql"
)

### Creating temp table to generate primary key ###
dim_atm.createOrReplaceTempView("dim_atm_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
### Writing the time dimension file to S3 ###
dim_atm.write.format('csv').option('header', 'true').save(
    's3://de-assignment-gov/dim_atm/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### DIM_CARD_TYPE ####

In [12]:
### Identifying the distinct DIM_CARD_TYPE using spark sql###

Dim_cardtype_distinct = spark.sql(
    " select distinct card_type from sourceFile_SQl")

### Creating temp table with spark Sql ###

Dim_cardtype_distinct.createOrReplaceTempView("Dim_cardtype_distinct_sql")

### Generating Row_number as ATM_ID using spark SQl ###

Dim_cardtype = spark.sql(
    " select ROW_NUMBER() OVER (ORDER BY card_type) as card_type_id, card_type from Dim_cardtype_distinct_sql"
)

### Creating temp table with spark Sql to use it in differen join ###

Dim_cardtype.createOrReplaceTempView("Dim_cardtype_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
### Writing the card type data to S3 ###
Dim_cardtype.write.format('csv').option('header', 'true').save(
    's3://de-assignment-gov/dim_card_type/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### DIM_DATE

In [14]:
### Created new Month_num field decoding the month from strings to number
from pyspark.sql.functions import when

sourceFile = sourceFile.withColumn(
    "Month_Num",
    when(sourceFile.month == 'January',
         '01').when(sourceFile.month == "February", "02").when(
             sourceFile.month == "March",
             "03").when(sourceFile.month == "April", "04").when(
                 sourceFile.month == "May",
                 "05").when(sourceFile.month == "June", "06").when(
                     sourceFile.month == "July",
                     "07").when(sourceFile.month == "August", "08").when(
                         sourceFile.month == "September",
                         "09").when(sourceFile.month == "October", "10").when(
                             sourceFile.month == "November",
                             "11").when(sourceFile.month == "December",
                                        "12").otherwise(sourceFile.month))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
### Created new day_Num field decoding the day from 0 to 00 , 1 to 01 etc...
sourceFile = sourceFile.withColumn(
    "Day_Num",
    when(sourceFile.day == '1', '01').when(sourceFile.day == "2", "02").when(
        sourceFile.day == "3", "03").when(sourceFile.day == "4", "04").when(
            sourceFile.day == "5", "05").when(sourceFile.day == "6", "06").when(
                sourceFile.day == "7",
                "07").when(sourceFile.day == "8",
                           "08").when(sourceFile.day == "9",
                                      "09").otherwise(sourceFile.day))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
### Created new Hour_Num field decoding the hour from 0 to 00 , 1 to 01 etc...
sourceFile = sourceFile.withColumn(
    "Hour_Num",
    when(sourceFile.hour == '1', '01').when(sourceFile.hour == "2", "02").when(
        sourceFile.hour == "0", "00").when(sourceFile.hour == "3", "03").when(
            sourceFile.hour == "4",
            "04").when(sourceFile.hour == "5",
                       "05").when(sourceFile.hour == "6",
                                  "06").when(sourceFile.hour == "7", "07").when(
                                      sourceFile.hour == "8", "08").when(
                                          sourceFile.hour == "9",
                                          "09").otherwise(sourceFile.hour))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
### Created new Complete_time_str conctaenting the above created colums

from pyspark.sql.functions import concat, col, lit

sourceFile = sourceFile.withColumn(
    'Complete_time_str',
    concat(col('year'), lit("-"), col('month_Num'), lit("-"), col('Day_Num'),
           lit(" "), col('Hour_Num'), lit(":"), lit("00"), lit(":"), lit("00")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
### Creating temp table with new fields
sourceFile.createOrReplaceTempView("sourceFile_SQl")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
### Date dimension table generation ###

Dim_Date_distinct = spark.sql(
    "select distinct Complete_time_str as full_date_time,year,month,day,hour,weekday  From sourceFile_SQl"
)

### Creating temp table with spark Sql ###

Dim_Date_distinct.createOrReplaceTempView("Dim_Date_distinct_sql")

### Generating Row_number as date_id using spark SQl ###

Dim_Date = spark.sql(
    " select ROW_NUMBER() OVER (ORDER BY full_date_time) as Date_id, full_date_time, year,month,day,hour,weekday from Dim_Date_distinct_sql"
)

### Creating temp table with spark Sql to use it in differen join ###

Dim_Date.createOrReplaceTempView("Dim_Date_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
### Writing the date dimension data to S3 ###
Dim_Date.write.format('csv').option('header', 'true').save(
    's3://de-assignment-gov/dim_date/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3. Fact table creation

In [21]:
### Hadling invalid data in the Message_text column for the message_Code 4017,4014,4002
sourceFile = sourceFile.withColumn(
    "Message_Txt_formatted",
    when(sourceFile.message_code == '4017', 'Suspected malfunction').when(
        sourceFile.message_code == '4014', 'Suspected malfunction').when(
            sourceFile.message_code == '4002',
            'Suspected malfunction').otherwise(sourceFile.message_text))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
### Creating temp table with new fields
sourceFile.createOrReplaceTempView("sourceFile_SQl")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
### Left joined with card type details and creation of temp tables
Card_type = spark.sql(
    "select a.*,b.card_type_id from sourceFile_SQl a  left join Dim_cardtype_sql b on a.card_type = b.card_type "
)

Card_type.createOrReplaceTempView("Card_type_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
### ### Left joined with weather loaction details and creation of temp table
weather_location = spark.sql(
    "select a.*, b.location_id as weather_loc_id from Card_type_sql a  left join Dim_Location_sql b on (a.atm_location = b.atm_location and a.atm_street_name = b.atm_street_name and a.atm_street_number = b.atm_street_number and  a.atm_zipcode = b.atm_zipcode and a.atm_lat = b.atm_lat and a.atm_lon = b.atm_lon) "
)
weather_location.createOrReplaceTempView("weather_location_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
### left joined with ATM temp table
atm_data = spark.sql(
    "select a.*, b.atm_id from weather_location_sql a  left join dim_atm_sql b on a.atm_id = b.atm_number and a.weather_loc_id=location_id"
)
atm_data.createOrReplaceTempView("atm_data_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
### left joined with Date temp table
time_fact_data = spark.sql(
    "select a.*, b.date_id from weather_location_sql a  left join Dim_Date_sql b on a.Complete_time_str = b.full_date_time"
)
time_fact_data.createOrReplaceTempView("time_fact_data_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
### Selecting only required columns ###
Fact_consolidate_columns = spark.sql(
    "select atm_id, weather_loc_id,date_id,card_type_id,atm_status,currency,service,transaction_amount,message_code,Message_Txt_formatted,rain_3h,clouds_all,weather_id,weather_main,weather_description from time_fact_data_sql"
)
Fact_consolidate_columns.createOrReplaceTempView("Fact_consolidate_columns_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
### Creating final table ###
Fact_consolidated = spark.sql(
    "Select ROW_NUMBER() OVER (ORDER BY a.date_id) as trans_id, a.* from Fact_consolidate_columns_sql a"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
t = Fact_consolidated.createOrReplaceTempView("Fact_consolidated_sql")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
### Writing the date dimension data to S3
Fact_consolidated.write.format('csv').option('header', 'true').save(
    's3://de-assignment-gov/fact_atm_trans/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…